# Analysis of the log-file
- how long did the pulse pattern take for each PIN
- What PINs have been evaluated?
  

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('PIN_unlock.log', sep=r'\\t', header = 0, engine='python')
# split column into multiple columns by delimiter 
df=df['logfile for PIN_unlock.py'].str.split(' - PIN - DEBUG - ', expand=True)
# rename columns 
df=df.rename(columns={0: "datetime", 1: "event"})
df['datetime']=pd.to_datetime(df['datetime'])
df['event'] = df['event'].str.replace('\'','')

df_split=df['event'].str.split('PIN: ', expand=True)
df_split=df_split.rename(columns={0: "event2", 1: "PIN", 2: "duration"})
df=df.join(df_split)

df_split=df['event2'].str.split('attempt nr: ', expand=True)
df_split=df_split.rename(columns={0: "event3", 1: "attempt"})
df=df.join(df_split)

df['PIN'] = df['PIN'].str.replace(' time last ','')
df['duration'] = df['duration'].str.replace(' sec','')
df=df.drop(['event2', 'event3'], axis=1)

df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
df['PIN'] = pd.to_numeric(df['PIN'], downcast='integer', errors='coerce')
df['attempt'] = pd.to_numeric(df['attempt'], downcast='integer',errors='coerce')

df.head(5)


## Visuals

In [ ]:
import matplotlib.pyplot as plt
#from matplotlib.finance import candlestick_ohlc
from matplotlib import style
import pandas as pd

style.use('classic')

def graph():
    #df = pd.DataFrame(candlestick_list)
    #ohlc = df[['date', 'open', 'high', 'low', 'close']]

    ax1 = plt.subplot2grid((6,1), (0,0), rowspan=5, colspan=1)
    ax2 = plt.subplot2grid((6,1), (5,0), rowspan=5, colspan=1, sharex=ax1)
    #candlestick_ohlc(ax1, ohlc.values, width=0.5, colorup='g', colordown='r')
    ax2.scatter(df['datetime'], df['PIN'])
    ax1.scatter(df['datetime'], df['duration'])
    plt.show()
graph()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x='datetime', y="duration")
fig.update_layout(height=300, width=400, title_text="PIN_unlock analysis")
fig.show()

In [ ]:

fig = px.scatter(df, x='datetime', y="PIN")
fig.update_layout(height=300, width=400, title_text="PIN_unlock analysis")
fig.show()

In [ ]:
fig = px.scatter(df, x='PIN', y="duration")
fig.update_layout(height=300, width=400, title_text="PIN_unlock analysis")
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1,
    subplot_titles=("Plot 1", "Plot 2")
)

fig.append_trace(
    go.Scatter(x=df['datetime'], y=df["PIN"]),
    row=1, col=1  
)

fig.append_trace(
    go.Scatter(x=df['datetime'], y=df["duration"]),
    row=2, col=1
)

fig.update_layout(height=600, width=400, title_text="PIN_unlock analysis")
fig.show()
